# Objectifs de ce Notebook
En application du TD3, je vais explorer ce que l’on pourrait faire avec les ontologies côté arXive, sur le modèle de ce que propose HAL. (Il y a donc in fine l’idée de proposer pour arXive une ontologie sommaire pour le domaine cs.AI. Du coup, je ne créé pas ici d’ontologie dans protégé, je le ferai plus tard pour correspondre au projet fil rouge (PFR) sur arXive.)
Dans le cadre du PFR, on pourrait vérifier/compléter nos données avec des requêtes dans des endpoints HAL (par exemple rechercher un doc archive dans HAL, ou des auteurs) ou d’autres.
Je vais créer une ontologie simple, directement avec owlready2, puis je vais la peupler avec SPARQLWrapper.
Dans un 3e temps je vais tenter d’explorer l’emploi du raisonneur, voire d’autres éléments.

## I. Création de l’ontologie dans PYTHON

### I. Création

In [1]:
from owlready2 import *
from SPARQLWrapper import *
import logging  # Pour le debug !
#from rdflib.namespace._FOAF import FOAF  # génère un warning


import sys


# Je vais modifier l'ontologie du TD, je la renomme. Elle sera basée sur FOAF puisque HAL l'utilise.
onto = get_ontology('https://cp.org/prepaFilRouge.owl#')
foaf = get_ontology('foaf.owl').load()
onto.imported_ontologies.append(foaf)

* Owlready2 * WARNING: DataProperty http://xmlns.com/foaf/0.1/nick belongs to more than one entity types: [owl.ObjectProperty, owl.DatatypeProperty]; I'm trying to fix it...
* Owlready2 * WARNING: DataProperty http://xmlns.com/foaf/0.1/aimChatID belongs to more than one entity types: [owl.ObjectProperty, owl.InverseFunctionalProperty, owl.DatatypeProperty, foaf.nick]; I'm trying to fix it...
* Owlready2 * WARNING: DataProperty http://xmlns.com/foaf/0.1/icqChatID belongs to more than one entity types: [owl.ObjectProperty, owl.InverseFunctionalProperty, owl.DatatypeProperty, foaf.nick]; I'm trying to fix it...
* Owlready2 * WARNING: DataProperty http://xmlns.com/foaf/0.1/jabberID belongs to more than one entity types: [owl.ObjectProperty, owl.InverseFunctionalProperty, owl.DatatypeProperty]; I'm trying to fix it...
* Owlready2 * WARNING: DataProperty http://xmlns.com/foaf/0.1/mbox_sha1sum belongs to more than one entity types: [owl.ObjectProperty, owl.InverseFunctionalProperty, owl.Datat

In [2]:
with onto:

    class Personne(foaf.Person): pass
    Personne.comment = ["Définition de la classe Personne qui spécialise foaf.Person"]

    class halPerson(Personne >> foaf.Person): pass
    halPerson.comment = [" link to foaf.Person from HAL, not necessary HalID !"]

    class halIdPerson(Personne >> foaf.Person, FunctionalProperty): pass
    halIdPerson.comment = [" link to halId"]

    class fullName(Personne >> str,FunctionalProperty): pass
    fullName.comment = ["Remplace name, déjà pris par owlready2"]


    class Publication(foaf.Document): pass
    Publication.comment = ["Définition de la classe Publication qui spéciffie foaf.Document"]
    Publication.comment.append("in fine, comme dans HAL on devrait pouvoir décrire le type d'un document dans arXive : préprint, affiche, these, livre, rapport")

    class halDocument(Publication >> foaf.Document, FunctionalProperty): pass  # TODO test pb en fonction de la version du doc ?
    halDocument.comment = ["lien vers le depot HAL"]

    class arXiveDocument(Publication >> foaf.Document,FunctionalProperty ) : pass
    arXiveDocument.comment = ["lien vers le depot arXive"]

    class ecrit(Personne >> Publication): pass

    class ecritPar(Publication >> Personne ):
        inverse = ecrit

    class maPublication(ObjectProperty):
        domaine = [foaf.Person |foaf.Document] # ce type de ligne, avec un ou n'est pas sauvegardé dans le fichier owl.
        range = [Publication]
    maPublication.comment = ["lien vers Publication, pour le raisonneur"]

    class maPersonne(ObjectProperty):
        domaine = [foaf.Person |foaf.Document]
        range = [Personne]
    maPersonne.comment = ["lien vers Personne, pour le raisonneur"]

    class monFoafEcritPar(ObjectProperty):
        domaine = [foaf.Document]
        range = [foaf.Person]
        inverse = foaf.publications
    monFoafEcritPar.comment =["fonction inverse de foaf.publications"]


    class Affiliation(foaf.Organization): pass

    class label(Affiliation >> str,FunctionalProperty): pass # DataProperty):
        #domaine = [Affiliation]
        #range = [str] # , FunctionalProperty): pass
    label.comment = [" Le prefLabel d'une organization"]

    class url(Affiliation >> str,FunctionalProperty): pass
        #domaine = [Affiliation]
        #range = str
    url.comment = ["L'url de l'organisazion, presque foaf.homepage mais sans imposer un foaf.Document "]

    class status(Affiliation>>str, FunctionalProperty):pass #DataProperty
    status.comment = ["Indique l'état actuel de l'organization, typiquement valid ou old"]

    class affiliationType (Affiliation >>str, FunctionalProperty ):pass
    affiliationType.comment = ["Caractérise l'organisation : institution, comme une école, Laboratory ou Researchteam au sein d'un labo"]
    affiliationType.comment.append("Pour le moment je ne gère pas la relation de composition des organizations")


    class member(foaf.member):
        domaine = [Affiliation]
        range = [foaf.Person]
    member.comment = ["duplication de foaf.member pour disposer des liens dans protégé"]

    AllDisjoint([Personne, Publication,Affiliation])

### I.2 Visualisation de l'ontologie dans python



In [3]:
# On peut explorer l'ontologie en python :
for i in foaf.classes():  #All classes
    print (i)
#for i in foaf.individuals():
#    print (i)
#properties(): #All properties
#object_properties(): #All object properties
#annotation_properties(): #All annotation properties
#disjoints(): #All pairwise disjoints (including pairwise distinct individuals and disjoint/distinct pairs)
#disjoint_classes()
#disjoint_properties() All pairwise disjoint properties (including disjoint pairs of properties)
#different_individuals() All pairwise distinct individuals (including distinct pairs of individuals)
#rules() All SWRL rules
#variables() All SWRL variables
#general_axioms() All general axioms

foaf.Agent
foaf.OnlineAccount
foaf.Document
foaf.Person
foaf.Image
foaf.Group
foaf.Organization


### I.3 Vérification  de l'ontologie dans python
Test rapide du modèle avec un raisonneur.
J'ai un message d'erreur d'owlready2, je vais donc sauver l'ontologie et la tester depuis protégé.

In [4]:
onto.save("prepaPFR_1.owl")

Aucune erreur n'est détectée sur ce fichier.

## II. Peuplement de l'ontologie

Je recherche tous les auteurs de HAL qui se nomment YOLAINE BOURDA. (En faisant abstraction des majuscules.)

## II.1 import d’auteurs Hal, première manière IRI déréférencable

Je ne vais rien dupliquer et conserver les IRI qui sont souvent déréférencable dans HAL.

In [5]:
#from owlready2.sparql.endpoint import *
sparql = SPARQLWrapper("http://sparql.archives-ouvertes.fr/sparql")
sparql.setTimeout(60)
query = """prefix foaf: <http://xmlns.com/foaf/0.1/>
select distinct ?s
where {
 ?s a foaf:Person; foaf:familyName ?nomFamille.
 #OPTIONAL {?s <http://www.openarchives.org/ore/terms/isAggregatedBy> ?o}
 ?s foaf:firstName ?prenom.
 #?s foaf:name ?name.

 FILTER (UCASE( ?prenom)="YOLAINE").
 FILTER (UCASE( ?nomFamille)="BOURDA").
}
LIMIT 100
"""

In [6]:
def extractUriValues (jsonSrc):  # Fonction pour extraire les URI du résultat d'une requête
    tabl = jsonSrc["results"]["bindings"]
    newResult = []
    for t in tabl:
        for k,v in t.items():
            if ('type' in v):
                if v['type']=='uri':
                    newResult.append(v['value'])
    return newResult

In [7]:
sparql.setQuery(query)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
tabFOAF = extractUriValues(results)
print(tabFOAF)

['https://data.archives-ouvertes.fr/author/46799', 'https://data.archives-ouvertes.fr/author/270747', 'https://data.archives-ouvertes.fr/author/yolaine-bourda', 'https://data.archives-ouvertes.fr/author/1420714', 'https://data.archives-ouvertes.fr/author/4635', 'https://data.archives-ouvertes.fr/author/1295368']


In [8]:
# Test manuel sur un cas
#px = foaf.Person()
#px.set_iri('https://data.archives-ouvertes.fr/author/46799')
#t= Personne(name=None,namespace=onto,halPerson = [px])


In [9]:
with onto:
    for i in tabFOAF:
        p = foaf.Person(namespace=onto)
        p.set_iri(i)
        # p.isDefinedBy = ["http://sparql.archives-ouvertes.fr/"]
        pers = Personne(halPerson = [p])
        p.maPersonne = [pers]

Je vérifie que ces individus sont bien dans l'ontologie

In [10]:
for i in onto.individuals():
    print (i,i.is_a)
    for prop in i.get_properties():
        for value in prop[i]:
            print(".%s == %s" % (prop.python_name, value))

author.46799 [foaf.Person]
.maPersonne == prepaFilRouge.personne1
prepaFilRouge.personne1 [prepaFilRouge.Personne]
.halPerson == author.46799
author.270747 [foaf.Person]
.maPersonne == prepaFilRouge.personne2
prepaFilRouge.personne2 [prepaFilRouge.Personne]
.halPerson == author.270747
author.yolaine-bourda [foaf.Person]
.maPersonne == prepaFilRouge.personne3
prepaFilRouge.personne3 [prepaFilRouge.Personne]
.halPerson == author.yolaine-bourda
author.1420714 [foaf.Person]
.maPersonne == prepaFilRouge.personne4
prepaFilRouge.personne4 [prepaFilRouge.Personne]
.halPerson == author.1420714
author.4635 [foaf.Person]
.maPersonne == prepaFilRouge.personne5
prepaFilRouge.personne5 [prepaFilRouge.Personne]
.halPerson == author.4635
author.1295368 [foaf.Person]
.maPersonne == prepaFilRouge.personne6
prepaFilRouge.personne6 [prepaFilRouge.Personne]
.halPerson == author.1295368


In [11]:
for i in onto.individuals():
    destroy_entity(i)

### II.2 import d'auteurs Hal, deuxième manière : duplication des données
Avec la requête, je vais préciser les éléments que je souhaite utiliser pour peupler l'ontologie

In [12]:
query="""prefix foaf: <http://xmlns.com/foaf/0.1/>
select distinct ?s ?name ?idHal
where {
 ?s a foaf:Person; foaf:familyName ?nomFamille.
 OPTIONAL {?s <http://www.openarchives.org/ore/terms/isAggregatedBy> ?idHal}
 ?s foaf:firstName ?prenom.
 ?s foaf:name ?name.

 FILTER (UCASE( ?prenom)="YOLAINE").
 FILTER (UCASE( ?nomFamille)="BOURDA").
}
LIMIT 100
"""

In [13]:
sparql.setTimeout(60)
sparql.setQuery(query)
sparql.setReturnFormat(CSV)
results = sparql.query().convert()

In [14]:
print (results)

b'"s","name","idHal"\n"https://data.archives-ouvertes.fr/author/46799","Yolaine Bourda","https://data.archives-ouvertes.fr/author/yolaine-bourda"\n"https://data.archives-ouvertes.fr/author/270747","Yolaine Bourda","https://data.archives-ouvertes.fr/author/yolaine-bourda"\n"https://data.archives-ouvertes.fr/author/yolaine-bourda","Yolaine Bourda",\n"https://data.archives-ouvertes.fr/author/1420714","Yolaine Bourda",\n"https://data.archives-ouvertes.fr/author/4635","Yolaine Bourda","https://data.archives-ouvertes.fr/author/yolaine-bourda"\n"https://data.archives-ouvertes.fr/author/1295368","Yolaine Bourda","https://data.archives-ouvertes.fr/author/yolaine-bourda"\n'


Je peux maintenant peupler l'ontologie, mais je dois procéder en deux temps car lorsque je créé la première foaf.Person, l'idHal (une foaf.Person également) n'existe pas encore.


In [15]:
tmpResult = results.decode().split('\n')[1:-1]# je n'utilise pas la ligne de titre ni la dernière (vide)
for l in tmpResult:
    #print(l)
    s,name,idHal=l.split(',')
    print(s,name,idHal)
    s = s.strip('"')
    name = name.strip('"')
    #print(s,name,idHal)
    p = foaf.Person(namespace=onto)
    p.set_iri(s)
    #p.isDefinedBy = ["http://sparql.archives-ouvertes.fr/"]
    pers = Personne(name=None,namespace=onto,halPerson = [p])
    pers.fullName = name
    p.maPersonne = [pers]

"https://data.archives-ouvertes.fr/author/46799" "Yolaine Bourda" "https://data.archives-ouvertes.fr/author/yolaine-bourda"
"https://data.archives-ouvertes.fr/author/270747" "Yolaine Bourda" "https://data.archives-ouvertes.fr/author/yolaine-bourda"
"https://data.archives-ouvertes.fr/author/yolaine-bourda" "Yolaine Bourda" 
"https://data.archives-ouvertes.fr/author/1420714" "Yolaine Bourda" 
"https://data.archives-ouvertes.fr/author/4635" "Yolaine Bourda" "https://data.archives-ouvertes.fr/author/yolaine-bourda"
"https://data.archives-ouvertes.fr/author/1295368" "Yolaine Bourda" "https://data.archives-ouvertes.fr/author/yolaine-bourda"


In [16]:
trueIdHal = "https://data.archives-ouvertes.fr/author/yolaine-bourda"

In [17]:
# Toutes les foaf.Person sont créées je peux stocker l'idHal (qui ici sera toujours le même).
trueIdHalPerson = onto.search_one(type = foaf.Person, iri = trueIdHal)
print(trueIdHalPerson,trueIdHalPerson.is_a)

author.yolaine-bourda [foaf.Person]


In [18]:
def myStrip(var:str):
    if var is None:
        return None
    var = var.strip('"')
    if len(var)== 0:
        return None
    return var

for l in tmpResult:
    #print(l)
    s,name,idHal=l.split(',')
    #print(s,name,idHal)
    s = myStrip(s)
    name = myStrip(name)
    idHal = myStrip(idHal)
    #print(s,name,idHal)
    if idHal is not None:
        #print(s, idHal)
        auteur = onto.search_one(type = foaf.Person, iri = s)
        pers = onto.search_one(type = onto.Personne,halPerson = [auteur])
        pers.halIdPerson = trueIdHalPerson


In [19]:
onto.save("prepaPFR_2.owl")  # Pour vérification dans Protégé

### II.3 import des structures HAL
Je vais importer dans l'ontologie les structures de mes personnes.

In [20]:
query="""PREFIX hal:<http://data.archives-ouvertes.fr/schema/>
PREFIX skos:<http://www.w3.org/2004/02/skos/core#>
PREFIX vcard:<http://www.w3.org/2006/vcard/ns#>
PREFIX classif:<http://www.w3.org/ns/org#>

select distinct ?structure ?label ?url ?status ?type
where
{
{{?agregat hal:person <https://data.archives-ouvertes.fr/author/46799>. } UNION
{?agregat hal:person <https://data.archives-ouvertes.fr/author/270747>. } UNION
{?agregat hal:person <https://data.archives-ouvertes.fr/author/yolaine-bourda>. } UNION
{?agregat hal:person <https://data.archives-ouvertes.fr/author/1420714>. } UNION
{?agregat hal:person <https://data.archives-ouvertes.fr/author/4635>. } UNION
{?agregat hal:person <https://data.archives-ouvertes.fr/author/1295368>. }}
?agregat hal:structure ?structure
optional {?structure skos:prefLabel ?label}
optional {?structure vcard:url ?url}
optional {?structure hal:status ?status}
optional {?structure classif:classification  ?type}
}
"""

In [21]:
sparql.setTimeout(60)
sparql.setQuery(query)
sparql.setReturnFormat(CSV)
results2 = sparql.query().convert()

In [22]:
#print (results2)
#TODO lien entre structures et membres !

In [23]:
tmpResult = results2.decode().split('\n')[1:-1]
with onto:
    for l in tmpResult:
        #print(l)
        structure,label,url,status,sType =l.split(',')
        structure = myStrip(structure)
        label = myStrip(label)
        url = myStrip(url)
        status = myStrip(status)
        sType = myStrip(sType)

        #print('s:',structure,' l:', label,' u:',url,' s:',status,' t:',sType)
        struc = Affiliation(namespace=onto)
        struc.set_iri(structure)
        struc.member = [trueIdHalPerson] # je donne un membre un peu arbitrairement ...
        if label is not None:
            struc.label = label
        if url is not None:
            struc.url = url
        if status is not None:
            struc.status = status
        if sType is not None:
            struc.affiliationType = sType


In [24]:
onto.save("prepaPFR_3.owl")

### II.4 import des documents depuis HAL
On va créer les publications. Je filtre les articles qui ont plus de 30 auteurs car je pense qu'il s'agit de données erronées.

In [25]:
query="""PREFIX dcterms:<http://purl.org/dc/terms/>
PREFIX doc:<https://data.archives-ouvertes.fr/document/>
PREFIX ore:<http://www.openarchives.org/ore/terms/>
PREFIX hal:<http://data.archives-ouvertes.fr/schema/>
PREFIX foaf:<http://xmlns.com/foaf/0.1/>

select  distinct ?document  ?NumCreators
where
{
?document dcterms:hasVersion ?version.
?version dcterms:creator ?creator.

{{?creator hal:person <https://data.archives-ouvertes.fr/author/46799>. } UNION
{?creator hal:person <https://data.archives-ouvertes.fr/author/270747>. } UNION
{?creator hal:person <https://data.archives-ouvertes.fr/author/yolaine-bourda>. } UNION
{?creator hal:person <https://data.archives-ouvertes.fr/author/1420714>. } UNION
{?creator hal:person <https://data.archives-ouvertes.fr/author/4635>. } UNION
{?creator hal:person <https://data.archives-ouvertes.fr/author/1295368>. }}

{
select ?version count(distinct ?o) as ?NumCreators
where {
?version dcterms:creator ?o.
}}
}
group by ?document ?NumCreators
HAVING ( ?NumCreators < 30)
"""

In [26]:
sparql.setQuery(query)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
tabPublications = extractUriValues(results)
#print(tabPublications)

In [27]:
with onto:
    for i in tabPublications:
        #print (i)
        d = foaf.Document(namespace=onto)
        d.set_iri(i)
        # p.isDefinedBy = ["http://sparql.archives-ouvertes.fr/"]
        p = Publication(name=None,namespace=onto,halDocument = d)
        d.maPublication = [p]

In [28]:
onto.save("prepaPFR_4.owl")

In [29]:
def queryAuthor(doc) :\
    return """PREFIX dcterms:<http://purl.org/dc/terms/>
    PREFIX doc:<https://data.archives-ouvertes.fr/document/>
    PREFIX hal:<http://data.archives-ouvertes.fr/schema/>

    #select ?version ?title ?pdf GROUP_CONCAT(?name,',') AS ?authors
    select distinct ?person
    where
    {<"""+doc+"""> dcterms:hasVersion ?version.
    ?version dcterms:creator ?creator.  #; ore:aggregates ?pdf.
    ?creator hal:person ?person.
    }"""

In [30]:
with onto:
    for i in tabPublications:
        #print(i)
        query = queryAuthor(i)
        #print(query)
        sparql.setQuery(query)
        sparql.setReturnFormat(JSON)
        results = sparql.query().convert()
        tabAuteurs = extractUriValues(results)
        foafDoc = onto.search_one(type = foaf.Document, iri = i)
        #print('fD:',foafDoc)
        for autIri in tabAuteurs:
            foafAuteur = onto.search_one(type = foaf.Person, iri = autIri)

            if foafAuteur is None:
                #print("création d'un auteur")
                foafAuteur = foaf.Person()
                foafAuteur.set_iri(autIri)

            #print ('fA:',foafAuteur.iri)
            foafAuteur.publications.append(foafDoc)
            foafDoc.monFoafEcritPar.append(foafAuteur) # Pour le raisoner

In [31]:
onto.save("prepaPFR_5.owl")

### III Utilisation du raisonneur

owlready2.JAVA_EXE=r'C:\Program Files\Java\jre1.8.0_351\bin\jave.exe'
provoque chez moi l'erreur FileNotFoundError. Il s'avère complexe de trouver de quel fichier il s'agit....

owlready2.JAVA_EXE=r'C:\Program Files (x86)\Java\jre1.8.0_261\bin\java.exe'
provoque chez moi l'erreur OwlReadyJavaError: Java error message is:
Error occurred during initialization of VM
Could not reserve enough space for 2048000KB object heap

Cette erreur peut être contournée en effectuant une modification dans le code owlready2.
Il s'agit dans reasoning.py de remplacer JAVA_MEMORY = 2000 par JAVA_MEMORY = 2000 (1500 ne passe pas).
Mais cela n'est pas très propre comme manière de faire.

En téléchargeant Java SE development kit 19.0.1 (JDK) je n'ai plus de problème !


In [32]:
# Change the following PATH if needed
owlready2.JAVA_EXE=r'c:\Program Files\Java\jdk-19\bin\java.exe'
#owlready2.JAVA_EXE='/usr/bin/java'


Je peux tester une partie de l'ontologie :

In [33]:
sync_reasoner([foaf])

* Owlready2 * Running HermiT...
    c:\Program Files\Java\jdk-19\bin\java.exe -Xmx2000M -cp C:\Users\tof\AppData\Local\Programs\Python\Python310\lib\site-packages\owlready2\hermit;C:\Users\tof\AppData\Local\Programs\Python\Python310\lib\site-packages\owlready2\hermit\HermiT.jar org.semanticweb.HermiT.cli.CommandLine -c -O -D -I file:///C:/Users/tof/AppData/Local/Temp/tmpyp1m8dw6
* Owlready2 * HermiT took 0.5138928890228271 seconds
* Owlready * Reparenting foaf.isPrimaryTopicOf: {owl.ObjectProperty, owl.InverseFunctionalProperty, foaf.page} => {foaf.page, owl.InverseFunctionalProperty}
* Owlready * Reparenting foaf.tipjar: {owl.ObjectProperty, foaf.page} => {foaf.page}
* Owlready * Reparenting foaf.weblog: {owl.ObjectProperty, owl.InverseFunctionalProperty, foaf.page} => {foaf.page, owl.InverseFunctionalProperty}
* Owlready * Reparenting foaf.img: {owl.ObjectProperty, foaf.depiction} => {foaf.depiction}
* Owlready * Reparenting foaf.homepage: {owl.ObjectProperty, owl.InverseFunctionalPr

Effectivement il y a quelques soucis mineurs : foaf.img est propriété dérivée de foaf.depiction. C'est donc forcement un ObjectProperty. (Pour être précis, img est une relation entre une foaf.person et une foaf.image qui "which are particularly representative". Je ne retrouve pas img dans protégé.)
La propriété nickname est à la fois une propriété et une data property.


In [34]:
sync_reasoner([foaf])

* Owlready2 * Running HermiT...
    c:\Program Files\Java\jdk-19\bin\java.exe -Xmx2000M -cp C:\Users\tof\AppData\Local\Programs\Python\Python310\lib\site-packages\owlready2\hermit;C:\Users\tof\AppData\Local\Programs\Python\Python310\lib\site-packages\owlready2\hermit\HermiT.jar org.semanticweb.HermiT.cli.CommandLine -c -O -D -I file:///C:/Users/tof/AppData/Local/Temp/tmpgb25wsqu
* Owlready2 * HermiT took 0.5385663509368896 seconds
* Owlready * (NB: only changes on entities loaded in Python are shown, other changes are done but not listed)


Après un deuxième appel au raisonneur les anomalies n'apparaissent plus. C'est bien la preuve que les inférences sont prises en compte.

In [35]:
with onto:
    try:
        #sync_reasoner()  # par HermiT
        #sync_reasoner(infer_property_values=True, debug=True, keep_tmp_file=True)
        #sync_reasoner_pellet()
        sync_reasoner_pellet(infer_property_values=True, infer_data_property_values=True, debug=True, keep_tmp_file=True)
        print("Ok, the ontology is consistent.")
    except OwlReadyInconsistentOntologyError:
        print("The ontology is inconsistent!")

* Owlready2 * Running Pellet...
    c:\Program Files\Java\jdk-19\bin\java.exe -Xmx2000M -cp C:\Users\tof\AppData\Local\Programs\Python\Python310\lib\site-packages\owlready2\pellet\antlr-3.2.jar;C:\Users\tof\AppData\Local\Programs\Python\Python310\lib\site-packages\owlready2\pellet\antlr-runtime-3.2.jar;C:\Users\tof\AppData\Local\Programs\Python\Python310\lib\site-packages\owlready2\pellet\aterm-java-1.6.jar;C:\Users\tof\AppData\Local\Programs\Python\Python310\lib\site-packages\owlready2\pellet\commons-codec-1.6.jar;C:\Users\tof\AppData\Local\Programs\Python\Python310\lib\site-packages\owlready2\pellet\httpclient-4.2.3.jar;C:\Users\tof\AppData\Local\Programs\Python\Python310\lib\site-packages\owlready2\pellet\httpcore-4.2.2.jar;C:\Users\tof\AppData\Local\Programs\Python\Python310\lib\site-packages\owlready2\pellet\jcl-over-slf4j-1.6.4.jar;C:\Users\tof\AppData\Local\Programs\Python\Python310\lib\site-packages\owlready2\pellet\jena-arq-2.10.0.jar;C:\Users\tof\AppData\Local\Programs\Python

* Owlready * Adding relation structure.2544 member author.yolaine-bourda
* Owlready * Adding relation structure.1050008 member author.yolaine-bourda
* Owlready * Adding relation structure.1050003 member author.yolaine-bourda
* Owlready * Adding relation structure.303397 member author.yolaine-bourda
* Owlready * Adding relation structure.400540 member author.yolaine-bourda
* Owlready * Adding relation structure.454174 member author.yolaine-bourda
* Owlready * Adding relation structure.411575 member author.yolaine-bourda
* Owlready * Adding relation structure.123689 member author.yolaine-bourda
* Owlready * Adding relation structure.21400 member author.yolaine-bourda
* Owlready * Adding relation structure.303397 label Ecole Supérieure d\'Electricité - SUPELEC (FRANCE)
Ok, the ontology is consistent.


* Owlready2 * Pellet took 0.9321086406707764 seconds
* Owlready2 * Warning: Cannot find new parent '9'
* Owlready2 * Warning: Cannot find new parent '9'
* Owlready2 * Warning: Cannot find new parent '9'
* Owlready * Reparenting prepaFilRouge.Affiliation: {foaf.Organization} => set()
* Owlready * Reparenting structure.303397: {prepaFilRouge.Affiliation} => {prepaFilRouge.Affiliation, foaf.Group}
* Owlready * Reparenting structure.123689: {prepaFilRouge.Affiliation} => {prepaFilRouge.Affiliation, foaf.Group}
* Owlready * Reparenting structure.454174: {prepaFilRouge.Affiliation} => {prepaFilRouge.Affiliation, foaf.Group}
* Owlready * Reparenting structure.2544: {prepaFilRouge.Affiliation} => {prepaFilRouge.Affiliation, foaf.Group}
* Owlready * Reparenting structure.400540: {prepaFilRouge.Affiliation} => {prepaFilRouge.Affiliation, foaf.Group}
* Owlready * Reparenting structure.1050003: {prepaFilRouge.Affiliation} => {prepaFilRouge.Affiliation, foaf.Group}
* Owlready * Reparenting structur

In [36]:
with onto:
    try:
        sync_reasoner()  # par HermiT
        #sync_reasoner(infer_property_values=True, debug=True, keep_tmp_file=True)
        #sync_reasoner_pellet()
        #sync_reasoner_pellet(infer_property_values=True, infer_data_property_values=True, debug=True, keep_tmp_file=True)
        print("Ok, the ontology is consistent.")
    except OwlReadyInconsistentOntologyError:
        print("The ontology is inconsistent!")

* Owlready2 * Running HermiT...
    c:\Program Files\Java\jdk-19\bin\java.exe -Xmx2000M -cp C:\Users\tof\AppData\Local\Programs\Python\Python310\lib\site-packages\owlready2\hermit;C:\Users\tof\AppData\Local\Programs\Python\Python310\lib\site-packages\owlready2\hermit\HermiT.jar org.semanticweb.HermiT.cli.CommandLine -c -O -D -I file:///C:/Users/tof/AppData/Local/Temp/tmpwmds7gss


Ok, the ontology is consistent.


* Owlready2 * HermiT took 0.48749256134033203 seconds
* Owlready * (NB: only changes on entities loaded in Python are shown, other changes are done but not listed)


HermiT ne trouve rien de plus.

In [37]:
onto.save("prepaPFR_6.owl")

In [38]:
with onto:
    # je n'arrive pas à donner à une ObjectProperty deux caractéristiques
    # je passe par une subproperties
    class baseIrreflexiveProperty (ObjectProperty, IrreflexiveProperty):pass

    class coEcrit(baseIrreflexiveProperty, SymmetricProperty):# todo préciser la def ?
        domain = [Personne]
        range = [Personne]
        #is_a = [     foaf.publications.some(foaf.Document) ]   # ecritPar(Publication >> Personne )

    class collegue(baseIrreflexiveProperty,SymmetricProperty):# todo a completer simpliste car on ne gère pas le temps
        domain = [Personne]
        range = [Personne]

# dans Protégé on voit que collegue est inféré comme etant son propre inverse.
# logique pour une relation symétrique.

In [39]:
# Avons-nous des classes inconsistantes ?
print(list(onto.world.inconsistent_classes()))

[]


In [40]:
onto.destroy()